In [1]:
from __future__ import division
from __future__ import print_function

import time
import argparse
import numpy as np
import math
import random
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

from utils import load_data, accuracy, normalize, load_polblogs_data
from models import GCN

In [2]:
class args:
    cuda = True
    fastmode = False
    seed = 20
#     seed = 123
    epochs = 200
    lr = 0.01
    weight_decay = 5e-4
    hidden = 16
    dropout = 0.5
    pert_num = 20
    L1 = 0.01
    evaluate_mode = "global_random"
    dataset = "polblogs"
    radius = 4


np.random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [3]:
if args.dataset == "polblogs":
    tmp_adj, features, labels, idx_train, idx_test = load_polblogs_data()
    print (sum(sum(tmp_adj)))
    print (tmp_adj.shape)
else:
    _, features, labels, idx_train, idx_val, idx_test, tmp_adj  = load_data(args.dataset)

num_classes = labels.max().item() + 1
# tmp_adj = tmp_adj.toarray()

adj = tmp_adj
adj = np.eye(tmp_adj.shape[0]) + adj
adj, _ = normalize(adj)
adj = torch.from_numpy(adj.astype(np.float32))

Selecting 1 largest connected components
33428.0
(1222, 1222)


/home/xiao/anaconda3/envs/pytorch/lib/python3.6/site-packages/scipy/sparse/_index.py:112: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [4]:
# Model and optimizer
model = GCN(nfeat=features.shape[1],
            nhid=args.hidden,
            nclass=num_classes,
            dropout=args.dropout)
optimizer = optim.Adam(model.parameters(),
                       lr=args.lr, weight_decay=args.weight_decay)

In [5]:
if args.cuda:
    model.cuda()
    features = features.cuda()
    adj = adj.cuda()
    labels = labels.cuda()
    idx_train = idx_train.cuda()
    if args.dataset != "polblogs":
        idx_val = idx_val.cuda()
    idx_test = idx_test.cuda()

In [6]:
def train(epoch):
    t = time.time()
    model.train()
    optimizer.zero_grad()
    x = Variable(adj, requires_grad=True)
    output = model(features, x)
    loss_train = F.nll_loss(output[idx_train], labels[idx_train])
    acc_train = accuracy(output[idx_train], labels[idx_train])
#     print ('output', output.size())
#     print ('labels', labels.size())
    loss_train.backward()

    optimizer.step()


    if args.dataset != "polblogs": 
        loss_val = F.nll_loss(output[idx_val], labels[idx_val])
        acc_val = accuracy(output[idx_val], labels[idx_val])
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'loss_val: {:.4f}'.format(loss_val.item()),
              'acc_val: {:.4f}'.format(acc_val.item()),
              'time: {:.4f}s'.format(time.time() - t))
    else:
        print('Epoch: {:04d}'.format(epoch+1),
              'loss_train: {:.4f}'.format(loss_train.item()),
              'acc_train: {:.4f}'.format(acc_train.item()),
              'time: {:.4f}s'.format(time.time() - t))

In [7]:
def test(adj_m):
    model.eval()
    output = model(features, adj_m)
    loss_test = F.nll_loss(output[idx_test], labels[idx_test])
    acc_test = accuracy(output[idx_test], labels[idx_test])
    print("Test set results:",
          "loss= {:.4f}".format(loss_test.item()),
          "accuracy= {:.4f}".format(acc_test.item()))
    return output

In [8]:
t_total = time.time()
for epoch in range(args.epochs):
    train(epoch)
print("Optimization Finished!")
print("Total time elapsed: {:.4f}s".format(time.time() - t_total))

# torch.save(model, './cora_gcn.pth')
# torch.save(model.state_dict(), 'cora_gcn.pkl')

# Testing
ori_output = test(adj)

Epoch: 0001 loss_train: 0.6825 acc_train: 0.6694 time: 0.4668s
Epoch: 0002 loss_train: 0.6769 acc_train: 0.7355 time: 0.0034s
Epoch: 0003 loss_train: 0.6698 acc_train: 0.7769 time: 0.0041s
Epoch: 0004 loss_train: 0.6644 acc_train: 0.6281 time: 0.0039s
Epoch: 0005 loss_train: 0.6676 acc_train: 0.7355 time: 0.0034s
Epoch: 0006 loss_train: 0.6382 acc_train: 0.8264 time: 0.0027s
Epoch: 0007 loss_train: 0.6431 acc_train: 0.7686 time: 0.0031s
Epoch: 0008 loss_train: 0.6060 acc_train: 0.8843 time: 0.0031s
Epoch: 0009 loss_train: 0.6115 acc_train: 0.7686 time: 0.0028s
Epoch: 0010 loss_train: 0.5864 acc_train: 0.9008 time: 0.0027s
Epoch: 0011 loss_train: 0.5667 acc_train: 0.9174 time: 0.0030s
Epoch: 0012 loss_train: 0.5501 acc_train: 0.9091 time: 0.0030s
Epoch: 0013 loss_train: 0.5329 acc_train: 0.9669 time: 0.0030s
Epoch: 0014 loss_train: 0.5235 acc_train: 0.9421 time: 0.0034s
Epoch: 0015 loss_train: 0.4962 acc_train: 0.9339 time: 0.0027s
Epoch: 0016 loss_train: 0.4642 acc_train: 0.9587 time: 

Epoch: 0180 loss_train: 0.0267 acc_train: 1.0000 time: 0.0037s
Epoch: 0181 loss_train: 0.0352 acc_train: 1.0000 time: 0.0032s
Epoch: 0182 loss_train: 0.0283 acc_train: 1.0000 time: 0.0032s
Epoch: 0183 loss_train: 0.0440 acc_train: 0.9917 time: 0.0039s
Epoch: 0184 loss_train: 0.0373 acc_train: 0.9917 time: 0.0036s
Epoch: 0185 loss_train: 0.0678 acc_train: 0.9917 time: 0.0030s
Epoch: 0186 loss_train: 0.0358 acc_train: 1.0000 time: 0.0028s
Epoch: 0187 loss_train: 0.0370 acc_train: 1.0000 time: 0.0028s
Epoch: 0188 loss_train: 0.0309 acc_train: 1.0000 time: 0.0028s
Epoch: 0189 loss_train: 0.0290 acc_train: 1.0000 time: 0.0028s
Epoch: 0190 loss_train: 0.0250 acc_train: 1.0000 time: 0.0028s
Epoch: 0191 loss_train: 0.0211 acc_train: 1.0000 time: 0.0027s
Epoch: 0192 loss_train: 0.0252 acc_train: 1.0000 time: 0.0031s
Epoch: 0193 loss_train: 0.0247 acc_train: 1.0000 time: 0.0028s
Epoch: 0194 loss_train: 0.0310 acc_train: 1.0000 time: 0.0028s
Epoch: 0195 loss_train: 0.0229 acc_train: 1.0000 time: 

In [9]:
def add_perturb(input_adj, idx, perturb):
    # (1-x)A + x(1-A)
#     input_adj = input_adj.toarray()

    x = np.zeros((input_adj.shape[0], input_adj.shape[1]))
    x[idx] = perturb  
    x[:,idx] = perturb
#     print ('x', x[idx])

    
#     x += np.transpose(x) #change the idx'th row and column
    x1 = np.ones((input_adj.shape[0], input_adj.shape[1])) - x
#     print ('x1', x1[idx])
    adj2 = np.ones((input_adj.shape[0], input_adj.shape[1])) - input_adj
#     print ('adj2', adj2[idx])

    for i in range(input_adj.shape[0]):      
        adj2[i][i] = 0

    perturbed_adj = np.multiply(x1, input_adj) + np.multiply(x, adj2)
    return perturbed_adj

In [10]:
def evaluate_attack(perturb):
    res = []
    # perturb = np.where(perturb>0.5, 1, 0)
    print ('perturb', perturb)
    new_pred = []
    for i in range(num_classes):
        new_pred.append(0)
    for k in idx_test:
#     for k in range(1):
#         print ('test node', k)
        innormal_x_p = add_perturb(tmp_adj, k, perturb)
#         print ('the perturbed conn', sum(innormal_x_p[k]))
#         innormal_x_p = np.where(innormal_x_p<0.5, 0, 1)

#         diff = innormal_x_p[k] - tmp_adj[k]
#         diff_idx = np.where(diff != 0 )
        
#         print ('diff_idx', diff_idx)
    #     one_idx = np.where(innormal_x_p[k]==1)[0]
    #     zero_idx = np.where(innormal_x_p[k]!=1)[0]
    #     total_idx = one_idx.shape[0] + zero_idx.shape[0]
    #     print ('total_idx', total_idx)
    #     print ('one_idx', one_idx)
    #     print ('corresponding perturb', perturb[one_idx])
    #     print (innormal_x_p[k][one_idx])
        x_p, degree_p = normalize(innormal_x_p + np.eye(tmp_adj.shape[0]))
        x_p = torch.from_numpy(x_p.astype(np.float32))
        x_p = x_p.cuda()
        output = model(features, x_p)
        new_pred[int(torch.argmax(output[k]))] += 1
        if int(torch.argmax(output[k])) == int(torch.argmax(ori_output[k])):
            res.append(0)
            print ('node {} attack failed'.format(k))
        else:
            res.append(1)
            print ('node {} attack succeed'.format(k))
    fooling_rate = float(sum(res)/len(res))
    print ('the current fooling rate is', fooling_rate)
    return fooling_rate, new_pred


In [11]:
def calculate_entropy(pred):
    h = 0
    all_pred = sum(pred)
    for i in range(num_classes):
        Pi = pred[i]/all_pred
        if Pi != 0:
            h -=  Pi* math.log(Pi)
    return h

In [12]:
new_pred = []
for i in range(num_classes):
    new_pred.append(0)
for k in idx_test:
    new_pred[int(torch.argmax(ori_output[k]))] += 1
entropy = calculate_entropy(new_pred)
print ('the entropy is', entropy)

the entropy is 0.6894199801171881


In [13]:
#evaluate the universal attack
if args.evaluate_mode == "universal":
    fool_res = []
    p_times = []
    all_entropy = []
    for i in range(10):
        perturb = np.array([float(line.rstrip('\n')) for line in open('./perturbation_results/{1}_xi{2}_epoch100/perturbation_{1}_{0}.txt'.format(i, args.dataset, args.radius))])
        perturb = np.where(perturb>0.5, 1, 0)
        pt = np.where(perturb>0)[0]
        if len(list(pt)) == 0:
            fool_res.append(0)
            p_times.append(0)
            continue
        print ('the perturbation is', pt)
        res, new_pred = evaluate_attack(perturb)
        print ('the prediction result is', new_pred)
        entropy = calculate_entropy(new_pred)
        fool_res.append(res)      
        p_times.append(len(list(pt)))
        print ('the perturbation times is', p_times)
        print ('the fooling rates are', fool_res)
        print ('the average fooling rates over 10 times of test is', sum(fool_res)/float(len(fool_res)))
        print ('the entropy is', entropy)
        all_entropy.append(entropy)
    print ('all the entropy values are', all_entropy)
    print ('the average entropy is', sum(all_entropy)/float(len(all_entropy)))
        
elif args.evaluate_mode == "global_random":
    #set this equal to the ceil of the number of anchor nodes computed by universal attack
    perturb_times = 6
    fool_res = []
    p_times = []
    for i in range(10):
#         perturb = np.array([float(line.rstrip('\n')) for line in open("perturbation.txt")])
#         perturb = np.where(perturb>0.5, 1, 0)
#         perturb_times = sum(perturb)
        # perturb = np.zeros(adj.shape[1])
        #the perturbation times of our universal perturbation
        # attack_index = list(np.random.choice(range(adj.shape[1]), perturb_times, replace = False))
        # perturb[attack_index] = 1
        prob = float(perturb_times / tmp_adj.shape[0])
        perturb = np.random.choice(2, tmp_adj.shape[0], p = [1-prob, prob])
        print ('the prob is', prob)
        pt = np.where(perturb>0)[0]
        print ('the perturbation is', pt)
        res, new_pred = evaluate_attack(perturb)
        fool_res.append(res)

        p_times.append(len(list(pt)))
        print ('the perturbation times is', p_times)
        print ('the fooling rates are', fool_res)
        print ('the average fooling rates over 10 times of test is', sum(fool_res)/float(len(fool_res)))

    print ('the average fooling rate with {} perturbation times is'.format(perturb_times), sum(fool_res)/float(len(fool_res)))
    
elif args.evaluate_mode == "limitted_random":
    
    perturb_times = 8
    fool_res = []
    p_times = []
    for i in range(10):
#         perturb = np.array([float(line.rstrip('\n')) for line in open("perturbation.txt")])
#         perturb = np.where(perturb>0.5, 1, 0)
#         perturb_times = sum(perturb)
        perturb = np.zeros(adj.shape[1])
        #the perturbation times of our universal perturbation
        attack_index = list(np.random.choice(range(adj.shape[1]), perturb_times, replace = False))
        perturb[attack_index] = 1
        pt = np.where(perturb>0)[0]
#         print ('the perturbation is', pt)
        res, new_pred = evaluate_attack(perturb)
        fool_res.append(res)

        p_times.append(len(list(pt)))
        print ('the perturbation times is', p_times)
        print ('the fooling rates are', fool_res)
        print ('the average fooling rates over 10 times of test is', sum(fool_res)/float(len(fool_res)))

    print ('the average fooling rate with {} perturbation times is'.format(perturb_times), sum(fool_res)/float(len(fool_res)))
    
elif args.evaluate_mode == "victim_attak":
    #the perturbation times of our universal perturbation
    perturb_time = 8 #set this equal to the ceil of the number of anchor nodes computed by universal attack
    fool_res = []
#     p_times = []
    for k in range(num_classes):
#     for k in range(4,6):
        each_fool_res = []
        idx = np.where(labels.cpu().numpy()==k)[0]
#         for i in range(1):
        for i in range(10):
            attack_index = list(np.random.choice(idx, perturb_time, replace = False))
            perturb = np.zeros(adj.shape[1])
            perturb[attack_index] = 1
            print ('perturbating by connecting to nodes of class', k)
            res, new_pred = evaluate_attack(perturb)
            each_fool_res.append(res)
            print ('the fooling rates of current class are', each_fool_res)
            avg_asr = sum(each_fool_res)/float(len(each_fool_res))
            print ('the average fooling rates over 10 times of test is', avg_asr)
        fool_res.append(avg_asr)
        print ('fool_res', fool_res)
    print ('the avg asr by connecting to each class of nodes is', fool_res)
        
elif args.evaluate_mode == "universal_delete":   
    
    all_fool = []
    for i in range(8, 9):
        fool_res = []
        for j in range(8):
            perturb = np.array([float(line.rstrip('\n')) for line in open('./perturbation_results/{1}_xi{2}_epoch100/perturbation_{1}_4.txt'.format(i, args.dataset, args.radius))])
            perturb = np.where(perturb>0.5, 1, 0)
            pt = np.where(perturb>0)[0]
            a = list(np.random.choice(range(0, pt.shape[0]), i, replace = False))
            perturb[pt[a]] = 0
            pt = np.where(perturb>0)[0]
            print ('the perturbation is', pt)
            res, new_pred = evaluate_attack(perturb)
            fool_res.append(res)      
            print ('the fooling rates are', fool_res)
            print ('the average fooling rates over 10 times of test is', sum(fool_res)/float(len(fool_res)))
        all_fool.append(sum(fool_res)/float(len(fool_res)))
          
    print ('all the fooling rate is', all_fool)

the prob is 0.004909983633387889
the perturbation is [330 547 609 649 950 955 972 995]
perturb [0 0 0 ... 0 0 0]
node 0 attack failed
node 1 attack failed
node 2 attack failed
node 3 attack succeed
node 4 attack succeed
node 5 attack failed
node 6 attack failed
node 7 attack failed
node 10 attack failed
node 11 attack failed
node 12 attack failed
node 14 attack succeed
node 15 attack failed
node 16 attack succeed
node 17 attack failed
node 18 attack failed
node 19 attack failed
node 20 attack failed
node 21 attack failed
node 22 attack succeed
node 23 attack failed
node 24 attack failed
node 26 attack succeed
node 27 attack failed
node 28 attack succeed
node 29 attack succeed
node 30 attack succeed
node 31 attack failed
node 32 attack failed
node 33 attack succeed
node 34 attack succeed
node 35 attack failed
node 36 attack failed
node 37 attack failed
node 38 attack failed
node 39 attack succeed
node 41 attack failed
node 42 attack failed
node 43 attack succeed
node 44 attack succeed
n

KeyboardInterrupt: 